## Dictionary 삽입 순서에 의존할 때는 조심하라

파이썬 3.5에서는 딕셔너리에 대해 이터레이션을 수행하면 키를 임의의 순서로 돌렸음
- 이렇게 됐던 이유는 내장 hash 함수와 파이썬 인터프리터가 시작할 때 초기화되는 seed 을 사용하는 해시 테이블 알고리즘으로 만들어졌기 때문이다.
- 그래서 매번 다름

3.6 부터는 dict 가 삽입 순서를 보존하도록 동작이 개선되었고, 3.7 부터는 언어 명세에 이 내용이 포함되었다.

In [2]:
baby_names = {
    'cat': 'kitten',
    'dog': 'puppy'
}

print(baby_names)

{'cat': 'kitten', 'dog': 'puppy'}


이로 인해 예전에는 키워드 인자(`**kwargs`)의 경우에는 순서가 뒤죽박죽인 것처럼 보였음 -> 이제는 그렇지 아니함

In [3]:
def my_func(**kwargs):
    for key, value in kwargs.items():
        print(f'{key} = {value}')

my_func(goose='gosling', kangaroo='joey')

goose = gosling
kangaroo = joey


삽입 순서 관련 동작이 항상 성립한다고 가정해서는 안 된다.
- 파이썬에서는 프로그래머가 리스트, 딕셔너리 등의 표준 프로토콜을 흉내 내는 커스텀 컨테이너 타입을 쉽게 정의할 수 있기 때문이다.
- 파이썬은 정적 타입 지정 언어가 아니기 때문에 대부분의 경우 코드는 엄격한 클래스 계층보다는 객체의 동작이 객체의 실질적인 타입을 결정하는 덕 타이핑에 의존하며, 이로 인해 가끔 어려운 함정에 빠질 수 있다.

덕 타이핑?
- "어떤 존재가 오리처럼 꽥꽥 소리를 내고, 오리처럼 보인다면 그것은 오리다"
- 라는 말로 흔히 설명되는 duck typing
- 동적 타입 지정의 일종으로, 객체가 실행 시점에 어떻게 행동하는지를 기준으로 객체의 타입을 판단하는 타입 지정 방식이다.
- 하지만 실제 행동을 모두 검증하기는 어려우므로 실제로는 객체가 제공하는 외부 인터페이스인 메서드와 애트리뷰트가 동일한지에 따라 타입이 같은지 결정한다.
- 실질적으로 이 말은 아무런 타이핑을 하지 않고 런타임에 객체가 제공하는 애트리뷰트와 메서드가 없는 경우에는 그냥 오류를 내겠다는 말과 같다.

In [5]:
votes = {
    'otter': 1281,
    'polar baer': 585,
    'fox': 863
}

In [10]:
def populate_ranks(votes, ranks):
    names = list(votes.keys())
    names.sort(key=votes.get, reverse=True)
    
    for i, name in enumerate(names, 1):
        ranks[name] = i

In [11]:
def get_winner(ranks):
    return next(iter(ranks))

In [12]:
ranks = {}
populate_ranks(votes, ranks)
print(ranks)

winner = get_winner(ranks)
print(winner)

{'otter': 1, 'fox': 2, 'polar baer': 3}
otter


이런 상황에서 프로그램의 요구 사항이 변경됐다고 하자. 결과를 보여줄 때 등수가 아니라 알파벳순으로 표시해야 한다.
- 이 경우에는 collections.abc 모듈을 사용해 딕셔너리와 비슷하지만 내용을 알파벳 순서대로 이터레이션해주는 클래스를 새로 정의할 수 있다.

In [17]:
from collections.abc import MutableMapping

class SortedDict(MutableMapping):
    def __init__(self):
        self.data = {}
        
    def __getitem__(self, key):
        return self.data[key]
    
    def __setitem__(self, key, value):
        self.data[key] = value
        
    def __delitem__(self, key):
        del self.data[key]
    
    def __iter__(self):
        keys = list(self.data.keys())
        keys.sort()
        
        for key in keys:
            yield key
    
    def __len__(self):
        return len(self.data)

SortedDict 는 표준 딕셔너리의 프로토콜을 지키므로, 앞에서 정의한 함수를 호출하면서 SortedDict 인스턴스를 표준 dict 위치에 사용해도 아무런 오류가 발생하지 않는다.
- 하지만 실행 결과는 요구 사항에 맞지 않는다.

In [18]:
sorted_ranks = SortedDict()
populate_ranks(votes, sorted_ranks)
print(sorted_ranks.data)

winner = get_winner(sorted_ranks)
print(winner)

{'otter': 1, 'fox': 2, 'polar baer': 3}
fox


여기서 문제는 get_winner 구현이 populate_ranks 의 삽입 순서에 맞게 딕셔너리를 이터레이션한다고 가정했다는 데 있다.

이 코드는 dict 대신 SortedDict 를 사용하므로 이 가정은 더 이상 성립하지 않는다.

따라서 우승 동물로는 득표수가 1등인 동물이 아니라 알파벳 순서로 맨 앞에 오는 동물인 fox가 반환된다.

해결 방법은 세 가지 있다.
- 첫 번째 방법은 ranks 딕셔너리가 어떤 특정 순서로 이터레이션된다고 가정하지 않고 get_winner 함수를 구현하는 것
- 두 번째 방법은 함수 맨 앞에 ranks의 타입이 우리가 원하는 타입인지 검사하는 코드를 추가하는 것
- 세 번째 방법은 타입 애너테이션(annotation)을 사용해서 get_winner에 전달되는 값이 딕셔너리와 비슷한 동작을 하는 MutableMapping 인스턴스가 아니라 dict 인스턴스가 되도록 강제하는 것

In [19]:
def get_winner(ranks):
    for name, rank in ranks.items():
        if rank == 1:
            return name

winner = get_winner(sorted_ranks)
print(winner)

otter


In [20]:
def get_winner(ranks):
    if not isinstance(ranks, dict):
        raise TypeError('dict 인스턴스가 필요합니다')

get_winner(sorted_ranks)

TypeError: dict 인스턴스가 필요합니다

In [ ]:
from typing import Dict, MutableMapping

def populate_ranks(votes: Dict[str, int],
                   ranks: Dict[str, int]) -> None:
    
    names = list(votes.keys())
    names.sort(key=votes.get, reverse=True)
    
    for i, rank in enumerate(ranks, 1):
        ranks[name] = i
        

def get_winner(ranks: Dict[str, int]) -> str:
    return next(iter(ranks))


class SortedDict(MutableMapping[str, int]):
    ...
    
# 이하 동일
# 이 방법은 dict 와 MutableMapping 타입의 차이를 올바로 감지해서 적절한 타입의 객체를 사용하지 않았을 때 오류를 발생시킨다. 이 해법은 정적 타입 안정성과 런타임 성능을 가장 잘 조합해준다.

딕셔너리와 비슷한 클래스를 조심스럽게 다루는 방법으로는 dict 인스턴스의 삽입 순서 보존에 의존하지 않고 코드를 작성하는 방법, 실행 시점에 명시적으로 dict 타입을 검사하는 방법, 타입 애너테이션과 정적 분석을 사용해 dict 값을 요구하는 방법이 있다.